# Module 4 Home assignment

Побудуйте класифікатор, який буде предбачати категорію запиту до служби 1551 з використанням тільки векторів слів (не використовувати інших фічі, такі як самі слова, нграми і т.д).



Вектори можна завантажити зі [сторінки проекту lang-uk](http://lang.org.ua/en/models/#anchor4) - можете обрати будь-які з векторів.



Запити знаходяться у архіві [1551.zip](https://github.com/vseloved/prj-nlp-2019/blob/master/tasks/1551.zip). Кожний файл в архіві має назву, яка відповідає категорії запитів, яку треба передбачати (загалом, 188 категорій), та містить запити у формі: id, текст запиту, 2 порожніх рядки. Наприклад у наступному фрагменті містяться 2 запити під номерами 3281192 та 2836792:
```

3281192

По вул.Шпака, на в’їзді до будинку №5 зі сторони буд. №3 відсутні захисні люки лівнєстоків. Ситуація є небезпечною для людей та автотранспорту. Прошу вжити необхідних заходів. Фото додаю.





2836792

Вітаю,



Мною подавалися заявки на виконання ремонту аварійних ділянок по вул. Сєченова та Коломийському провулку у м. Києві. Заявки № Я-2800 від 01.07.2013 та № Я-3167 від 23.07.2013. Мені було надано відповідь, що ремонт буде виконано при отриманні фінансування. З моменту моїх звернень пройшло більше року, було затверджено бюджет Києва на 2014 рік, проте ремонтом дорожнього покриття на вул. Сєченова так ніхто і не займався. 

На сьогодні більшість отворів дощових колекторів по вул. Сєченова не укомплектовані решітками, що ставить під загрозу безпеку пересування транспортних засобів та пішоходів, та не відповідає вимогам ДСТУ.

На виконання вимог ДСТУ 3587-97. Безпека дорожнього руху. Автомобільні дороги, вулиці та залізничні переїзди. Вимоги до експлуатаційного стану ПОВТОРНО прошу:

1. Укомплектувати отвори дощових колекторів по вул. Сєченова металевими зливними решіткам замість дощок (фото додаються).



Заздалегідь вдячний.

```



Можете використайте фільтрацію мови, щоб відібрати тільки україномовні запити.


Для початку, завантажуємо вектори:

In [1]:
import numpy as np
import gensim
import utils
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neural_network import MLPClassifier

In [2]:
ubercorpus = 'ubercorpus.lowercased.tokenized.word2vec.300d'

In [3]:
!wget http://lang.org.ua/static/downloads/models/ubercorpus.lowercased.tokenized.word2vec.300d.bz2

--2019-08-04 13:01:36--  http://lang.org.ua/static/downloads/models/ubercorpus.lowercased.tokenized.word2vec.300d.bz2
Resolving lang.org.ua (lang.org.ua)... 95.216.74.77
Connecting to lang.org.ua (lang.org.ua)|95.216.74.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 502679218 (479M) [application/octet-stream]
Saving to: ‘ubercorpus.lowercased.tokenized.word2vec.300d.bz2’

ubercorpus.lowercas 100%[===================>] 479.39M  1.70MB/s    in 4m 30s  

2019-08-04 13:06:06 (1.78 MB/s) - ‘ubercorpus.lowercased.tokenized.word2vec.300d.bz2’ saved [502679218/502679218]



In [ ]:
!bzip2 -d ubercorpus.lowercased.tokenized.word2vec.300d.bz2

Завантажуємо датасет

In [ ]:
!wget https://github.com/vseloved/prj-nlp-2019/raw/master/tasks/1551.zip

In [ ]:
!unzip 1551.zip

In [4]:
data, targets_dict = utils.load_dataset_from_dir("1551")

In [5]:
data[:5]

[('Хочу Вас спитати, маю я право на субсидію чи ні!\nВ квартирі прописані я та дочка. Батько наш помер у 2014. Він мав інваліда війни 3 групи.Пільги як вдова померлого я оформила. В нас трикімнатна крартира. Кожен місяць рахунки більші за мою пенсію.Дочка навчається. Доходів крім моєї пенсії немає.\n  \n\nз повагою, Гончаренко Олена Віталіївна\nконт.273-56-71; 096-118-49-88; 099-143-30-12\n\n\n',
  0),
 ('Питання про надання субсидій. Чому ходять чутки, що при продажі квартири, на яку було оформлено субсидію, з власника квартири буде стягнуто всю суму нарахованої субсидії за весь час отримання субсидії? Чому про це ніде не повідомляється? Прошу надати вичерпну відповідь.\n\n\n',
  0),
 ('Цього року я подала поштою необхідні документи (заяву, декларацію) для призначення субсидії на послуги ЖКГ до управління соцзахитсу населення Дніпровської РДА м.Києва. На сьогоднішній день відповіді про призначення субсидії чи відмову в призначенні не отримала, незважаючи на те, що передбачені законода

In [6]:
targets_dict

{'Розрахунки--нарахування-та-перерахунок-субсидій': 0,
 'Встановлення-огородження-зеленої-зони': 1,
 'Не-працює-вантажний--ліфт': 2,
 'Забруднення-повітря-від-невідомих-джерел': 3,
 'Утримання-підвалів--колясочних-технічних-поверхів': 4,
 'Правила-торговельного-обслуговування-населення': 5,
 'Питання--що-стосуються-завершення-опалювального-сезону': 6,
 'Незадовільний-стан-опори-для-освітлення': 7,
 'Порушення-правил-дорожнього-руху': 8,
 'Бажаючі-отримати--Картки-киянина--КК--': 9,
 'Раціоналізаторські-пропозиції': 10,
 'Спалювання-сміття--листя--гілок--відходів--тощо-': 11,
 'Ремонт-та-заміна-вхідних-дверей': 12,
 'Паркування-авто-на-зеленій-зоні-прибудинкової-території': 13,
 'Перерахунок-плати-за-незадовільну-температуру-ГВП': 14,
 'Перевірка-дозвільної-документації--демонтаж-окремо-розташованих-гаражів': 15,
 'Демонтаж-рекламних-конструкцій-і-вивісок': 16,
 'Незручності-від-розташування-холодильників--лотків-біля-кіосків': 17,
 'Паркування-авто-у-місцях-загального-користування': 18

1. У якості безйлайну використайте класифікатор kNN, який буде шукати найближчі за косинусною відстанню вектори запитів, утворені сумуванням векторів всіх слів запиту.

Перетворимо кожен текст у нормальний вектор, використовуючи гтовий ubercorpus

In [7]:
model = gensim.models.KeyedVectors.load_word2vec_format(ubercorpus)

In [8]:
vector_data = utils.dataset2vector(data, model)

/Volumes/data/Data Science/NLP/Module4/Data_Science-NLP-N-Grams/utils.py:87: RuntimeWarning: invalid value encountered in true_divide
  v = vector / np.linalg.norm(vector)


In [9]:
X_train, X_test, y_train, y_test = utils.get_train_and_test_data(vector_data)

In [10]:
knn_classifier = KNeighborsClassifier(metric='cosine')

In [11]:
knn_classifier.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='cosine',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [12]:
y_pred = knn_classifier.predict(X_test)

In [13]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.56      0.50      0.53        20
           1       0.11      0.17      0.13        70
           2       0.21      0.37      0.27        54
           3       0.24      0.44      0.31        25
           4       0.15      0.32      0.20       142
           5       0.29      0.25      0.27        16
           6       0.39      0.38      0.38        32
           7       0.15      0.18      0.16        28
           8       0.24      0.31      0.27        16
           9       0.41      0.50      0.45        14
          10       0.09      0.11      0.10        19
          11       0.07      0.12      0.09         8
          12       0.30      0.33      0.31        40
          13       0.03      0.10      0.05        10
          14       0.08      0.14      0.10        22
          15       0.05      0.12      0.07        16
          16       0.41      0.40      0.40       110
          17       0.08    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


В репорті наведеному вижче, можемо побачити F1 score даної моделі - 0.36

2. Розробіть покращений класифікатор, для якого використайте іншу модель класифікації, яка би присвоювала ваги окремим елементам вектора (наприклад, Logistic Regression, SVM, Random Forest). Також можете зробити додаткові покращення, такі як:

    - здійснювати попередню фільтрацію і відбирати тільки частину векторів для представлення

    - використовувати лематизацію та інші форми нормалізації

    - робити зваження векторів

    - додатково використовувати інші способи побудувати вектор документу (наприклад, TF-IDF, LDA, ...)



Для початку використаємо Random Forest модель з тими ж самими даними

In [14]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=25, random_state=42)

In [15]:

rf_classifier.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [16]:
y_pred = rf_classifier.predict(X_test)
y_pred

array([ 75, 158,  75, ...,  75, 158,  75])

In [17]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.10      0.18        20
           1       0.91      0.14      0.25        70
           2       0.82      0.26      0.39        54
           3       1.00      0.08      0.15        25
           4       0.58      0.21      0.31       142
           5       1.00      0.12      0.22        16
           6       0.87      0.41      0.55        32
           7       0.00      0.00      0.00        28
           8       1.00      0.25      0.40        16
           9       1.00      0.29      0.44        14
          10       1.00      0.11      0.19        19
          11       0.00      0.00      0.00         8
          12       1.00      0.30      0.46        40
          13       0.00      0.00      0.00        10
          14       1.00      0.14      0.24        22
          15       1.00      0.19      0.32        16
          16       0.81      0.31      0.45       110
          17       1.00    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


З вище наведеного репорту видно, що Random Forest модель дало трішки кращий F1 scrore порівняно з KNeighborsClassifier (0.39 > 0.36).

Далі спробуємо виключити стоп слова

Завантажуємо список стоп слів

In [ ]:
!wget https://gist.githubusercontent.com/kissarat/bec2bb727c9fb520043a/raw/84cb14b1f653fd027aed769810070c02e71c157f/ukrainian-stopwords.txt

In [18]:
stop_words =  utils.read_stop_words('ukrainian-stopwords.txt')

In [19]:
vector_data = utils.dataset2vector(data, model, stop_words)

/Volumes/data/Data Science/NLP/Module4/Data_Science-NLP-N-Grams/utils.py:87: RuntimeWarning: invalid value encountered in true_divide
  v = vector / np.linalg.norm(vector)


In [20]:
X_train, X_test, y_train, y_test = utils.get_train_and_test_data(vector_data)

In [21]:
rf_classifier.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [22]:
y_pred = rf_classifier.predict(X_test)
y_pred

array([ 30, 134,  54, ...,  30,  36, 107])

In [23]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.07      0.13        14
           1       1.00      0.08      0.14        65
           2       0.91      0.31      0.47        64
           3       1.00      0.15      0.27        26
           4       0.51      0.26      0.35       134
           5       1.00      0.11      0.19        19
           6       1.00      0.31      0.47        26
           7       1.00      0.09      0.16        23
           8       0.50      0.28      0.36        18
           9       1.00      0.27      0.43        11
          10       1.00      0.09      0.17        11
          11       1.00      0.36      0.53        14
          12       0.75      0.34      0.47        35
          13       0.00      0.00      0.00        11
          14       0.67      0.10      0.17        21
          15       1.00      0.40      0.57        10
          16       0.65      0.29      0.40        98
          17       0.00    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Як видно з репорту вище, F1 score з урахуванням виключених стоп слів трішки покращилось (0.40 < 0.39). 

### TF-IDF vectorize

Спробуємо перерахувати вектори, використовуючи TF-IDF

In [24]:
vector_data = utils.dataset2vector(data, model, stop_words, use_tf_idf_weights=True)

/Volumes/data/Data Science/NLP/Module4/Data_Science-NLP-N-Grams/utils.py:87: RuntimeWarning: invalid value encountered in true_divide
  v = vector / np.linalg.norm(vector)


In [25]:
X_train, X_test, y_train, y_test = utils.get_train_and_test_data(vector_data)

In [26]:
rf_classifier.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [27]:
y_pred = rf_classifier.predict(X_test)
y_pred

array([158, 144, 158, ...,  30, 158, 158])

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.36      0.20      0.26        20
           1       0.24      0.13      0.17        47
           2       0.76      0.38      0.51        50
           3       0.50      0.10      0.16        31
           4       0.46      0.14      0.22       134
           5       0.00      0.00      0.00        11
           6       0.71      0.43      0.53        28
           7       0.33      0.06      0.10        33
           8       0.33      0.33      0.33         6
           9       0.67      0.15      0.25        13
          10       0.00      0.00      0.00        19
          11       0.67      0.15      0.25        13
          12       0.50      0.28      0.36        29
          13       0.00      0.00      0.00        19
          14       0.00      0.00      0.00        19
          15       1.00      0.13      0.24        15
          16       0.57      0.26      0.36       110
          17       0.40    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


З репорту наведеного вище, ми бачимо що як на диво зважені коефіцієнти tf-idf  (домножадись на попередній вектор) дали гірший результат. за попередній (0.29 < 0.40)

3. Розробіть класифікатор на основі нейромережі: або feed-forward з використанням агрегованого вектору, або LSTM (чи BiLSTM), що проходить по всьому тексту. 

In [29]:
vector_data = utils.dataset2vector(data, model, stop_words, use_tf_idf_weights=False)

/Volumes/data/Data Science/NLP/Module4/Data_Science-NLP-N-Grams/utils.py:87: RuntimeWarning: invalid value encountered in true_divide
  v = vector / np.linalg.norm(vector)


In [30]:
X_train, X_test, y_train, y_test = utils.get_train_and_test_data(vector_data)

In [31]:
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15, 2), random_state=1)

In [32]:
clf.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(15, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [33]:
y_pred = clf.predict(X_test)

In [34]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        17
           1       0.00      0.00      0.00        47
           2       0.00      0.00      0.00        55
           3       0.00      0.00      0.00        22
           4       0.08      0.10      0.09       112
           5       0.00      0.00      0.00        11
           6       0.00      0.00      0.00        38
           7       0.00      0.00      0.00        30
           8       0.00      0.00      0.00        14
           9       0.00      0.00      0.00        19
          10       0.00      0.00      0.00        18
          11       0.00      0.00      0.00         8
          12       0.00      0.00      0.00        33
          13       0.00      0.00      0.00         9
          14       0.00      0.00      0.00        21
          15       0.00      0.00      0.00        13
          16       0.00      0.00      0.00        91
          17       0.00    

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Як видно з вище наведеного репорту, звичайна feed forward нейромережа з використанням попередньо згенерованого вектора (сума векторів слів в тектсі) не дала покращення, а навпаки F1 score значно погіршився (0.18 < 0.38). Необхідно досліджувати інші підходити з використанням нейромереж типу LSTM, CRF - ймовірно дадуть кращий результат за Random Forest.